In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import os

In [45]:
def loadNpy(filename):
    with open(os.getcwd() + "/train-val-test/"+ filename, "rb") as f: return np.load(f)

db = "db14/window-size-10/lag2/all-features/"
data_type = ""

In [46]:
DIR = os.getcwd() + "/train-val-test/" + db

In [62]:
X_train, X_val, X_test = loadNpy(db + f"pca-X_train.npy"), loadNpy(db + f"pca-X_val.npy"), loadNpy(db + f"pca-X_test.npy")
y_train, y_val, y_test = loadNpy(db + f"y{data_type}_train.npy"), loadNpy(db + f"y{data_type}_val.npy"), loadNpy(db + f"y{data_type}_test.npy")

In [48]:
def npy_to_tensor(data, l=False):
    if not l:
        data = torch.from_numpy(data).float()
    else:
        data = torch.from_numpy(data).type(torch.LongTensor)
    return data

In [49]:
X_train, X_val, X_test = npy_to_tensor(X_train), npy_to_tensor(X_val), npy_to_tensor(X_test)
y_train, y_val, y_test = npy_to_tensor(y_train,1), npy_to_tensor(y_val,1), npy_to_tensor(y_test,1)

In [50]:
batch_size = 128

In [51]:
train_dataset = TensorDataset(X_train, y_train)

val_dataset = TensorDataset(X_val, y_val)

test_dataset = TensorDataset(X_test, y_test)

# Data loader
train_iterator = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

val_iterator = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

test_iterator = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [44]:
torch.save(train_dataset, DIR + "train-dataset")
torch.save(val_dataset, DIR + "val-dataset")
torch.save(test_dataset, DIR + "test-dataset")

In [8]:
X_train.shape

torch.Size([17370, 1, 28, 28])

In [9]:
X_val.shape

torch.Size([5791, 1, 28, 28])

In [10]:
X_train.dtype

torch.float32

# CNN

In [52]:
loaders = {
    'train': train_iterator,
    'val': val_iterator,
    'test': test_iterator
}

In [53]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(32, 64, 5, 1, 2),     
            nn.ReLU()               
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(64 * 7 * 7, 2)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

In [54]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (out): Linear(in_features=3136, out_features=2, bias=True)
)


In [55]:
loss_func = nn.CrossEntropyLoss()   
loss_func

CrossEntropyLoss()

In [56]:
optimizer = optim.Adam(cnn.parameters(), lr = 0.001)   
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [57]:
from torch.autograd import Variable

num_epochs = 20

def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                pass
        pass
    pass

train(num_epochs, cnn, loaders)

Epoch [1/20], Step [100/136], Loss: 0.3013
Epoch [2/20], Step [100/136], Loss: 0.2358
Epoch [3/20], Step [100/136], Loss: 0.2054
Epoch [4/20], Step [100/136], Loss: 0.1796
Epoch [5/20], Step [100/136], Loss: 0.1633
Epoch [6/20], Step [100/136], Loss: 0.1544
Epoch [7/20], Step [100/136], Loss: 0.1469
Epoch [8/20], Step [100/136], Loss: 0.1408
Epoch [9/20], Step [100/136], Loss: 0.1341
Epoch [10/20], Step [100/136], Loss: 0.1289
Epoch [11/20], Step [100/136], Loss: 0.1245
Epoch [12/20], Step [100/136], Loss: 0.1148
Epoch [13/20], Step [100/136], Loss: 0.1128
Epoch [14/20], Step [100/136], Loss: 0.1090
Epoch [15/20], Step [100/136], Loss: 0.1045
Epoch [16/20], Step [100/136], Loss: 0.1005
Epoch [17/20], Step [100/136], Loss: 0.1019
Epoch [18/20], Step [100/136], Loss: 0.0974
Epoch [19/20], Step [100/136], Loss: 0.0933
Epoch [20/20], Step [100/136], Loss: 0.0922


In [93]:
def eval_cnn(data):
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders[data]:
            test_output, last_layer = cnn(images)
            print(test_output)
            print(torch.max(test_output, 1))
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            print(pred_y)
            total += len(labels)
            correct += (pred_y == labels).sum().item()
            pass
        print(f'{data} Accuracy of the model on the {total} {data} images: %.2f' % (correct/total))
    pass

In [94]:
eval_cnn("val")
eval_cnn("test")

tensor([[-9.6650,  9.5735],
        [ 3.1151, -3.1747],
        [ 2.0757, -2.1471],
        [ 1.9262, -1.9846],
        [ 2.5305, -2.6473],
        [ 2.9044, -2.9448],
        [ 0.8062, -0.8657],
        [ 4.1487, -4.2865],
        [ 3.9494, -4.0399],
        [ 2.7676, -2.8498],
        [ 2.4387, -2.5178],
        [ 4.9396, -5.0032],
        [ 1.6462, -1.6834],
        [ 3.9249, -3.9493],
        [ 0.0262, -0.0644],
        [ 3.6955, -3.7681],
        [ 4.9902, -5.0320],
        [ 0.8831, -0.8859],
        [ 3.7082, -3.7858],
        [ 3.8042, -3.7462],
        [ 2.9105, -2.9779],
        [ 3.5501, -3.5867],
        [ 3.1973, -3.2008],
        [ 1.2524, -1.2813],
        [ 2.8187, -2.8955],
        [ 4.6155, -4.7006],
        [ 4.5204, -4.5847],
        [ 1.1573, -1.1747],
        [ 4.2369, -4.3151],
        [ 1.7413, -1.9091],
        [ 2.5561, -2.6522],
        [ 3.6872, -3.7175],
        [ 2.3887, -2.4400],
        [ 0.7712, -0.7168],
        [ 3.9243, -3.9911],
        [ 2.7260, -2

tensor([[ 2.3740, -2.4863],
        [ 1.1286, -1.2280],
        [ 3.8544, -3.9442],
        [ 2.7094, -2.7460],
        [ 1.1695, -1.2229],
        [ 1.8159, -1.9040],
        [ 1.6868, -1.7240],
        [ 3.2631, -3.3081],
        [ 3.3991, -3.4696],
        [ 1.0760, -1.1272],
        [-1.6382,  1.5671],
        [ 5.0846, -5.1855],
        [ 4.4211, -4.4547],
        [ 2.1927, -2.1938],
        [ 3.0087, -3.0421],
        [ 4.4687, -4.5053],
        [ 3.0309, -3.0730],
        [-1.1322,  1.1391],
        [ 3.5750, -3.6651],
        [ 3.4859, -3.5519],
        [ 2.1800, -2.1641],
        [-0.2890,  0.1700],
        [ 2.5453, -2.6267],
        [-8.5939,  8.5951],
        [ 3.9300, -3.9419],
        [ 2.4322, -2.4978],
        [ 3.3542, -3.4150],
        [ 3.4070, -3.4748],
        [-2.3850,  2.4247],
        [ 2.0833, -2.1508],
        [ 4.0076, -4.0715],
        [ 3.1536, -3.2562],
        [ 5.0648, -5.0555],
        [ 2.9164, -2.9923],
        [ 2.4645, -2.5179],
        [ 3.0830, -3

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0])
tensor([[ 8.1846e-01, -8.6396e-01],
        [ 2.4826e+00, -2.4669e+00],
        [-3.5587e+00,  3.5563e+00],
        [ 1.6525e+00, -1.7205e+00],
        [ 1.8303e+00, -1.9759e+00],
        [ 1.7222e+00, -1.8083e+00],
        [ 3.5539e+00, -3.6705e+00],
        [ 3.5786e+00, -3.6024e+00],
        [ 3.0175e+00, -3.0845e+00],
        [ 4.2637e+00, -4.2858e+00],
        [ 2.5303e+00, -2.6081e+00],
        [ 4.0887e+00, -4.1443e+00],
        [ 4.2667e+00, -4.3425e+00],
        [ 3.1659e+00, -3.2465e+00],
        [ 6.1040e+00, -6.1497e+00],
        [ 4.2247e-01, -4.49

tensor([[ 3.6824, -3.7631],
        [ 6.5832, -6.6078],
        [ 1.9944, -2.0559],
        [ 2.5764, -2.6374],
        [ 3.0481, -3.1278],
        [ 3.8216, -3.8898],
        [ 3.3465, -3.3980],
        [ 4.2913, -4.4007],
        [ 1.0369, -1.0979],
        [ 3.2867, -3.3710],
        [ 3.9256, -4.0488],
        [ 4.6197, -4.6793],
        [-7.1316,  7.0603],
        [ 2.2503, -2.2918],
        [ 3.3104, -3.3834],
        [ 4.2364, -4.1927],
        [ 3.9094, -3.9835],
        [ 3.1773, -3.2303],
        [ 4.0468, -4.0564],
        [ 2.5369, -2.6121],
        [-0.1387,  0.2223],
        [ 4.3632, -4.4268],
        [ 2.5455, -2.5838],
        [ 4.8595, -4.9149],
        [ 3.9008, -3.9695],
        [ 2.1258, -2.1935],
        [ 2.4506, -2.5430],
        [ 0.8081, -0.8509],
        [ 3.4477, -3.5240],
        [-7.3436,  7.3148],
        [ 7.1404, -7.2036],
        [ 2.4382, -2.5266],
        [ 2.8888, -2.9737],
        [ 2.6754, -2.7208],
        [ 3.1416, -3.1981],
        [ 2.6948, -2

In [64]:
import onnx

In [68]:
cnn.eval()
dummy_input = torch.randn(1, 1, 28, 28)
input_names = [ "actual_input" ]
output_names = [ "output" ]
model = cnn

In [69]:
torch.onnx.export(model, 
                  dummy_input,
                  "cnn.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )

In [41]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [69]:
class VGG(nn.Module):
    def __init__(self, features, output_dim):
        super().__init__()

        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

In [114]:
vgg11_config = [6, 'M', 12, 'M', 24, 24, 'M']#[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512,
                512, 'M']

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']

In [115]:
def get_vgg_layers(config, batch_norm):

    layers = []
    in_channels = 3

    for c in config:
        assert c == 'M' or isinstance(c, int)
        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size=2)]
        else:
            conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = c

    return nn.Sequential(*layers)

In [116]:
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)

In [117]:
OUTPUT_DIM = 2

model = VGG(vgg11_layers, OUTPUT_DIM)

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2

## TRAINING

In [118]:
START_LR = 1e-7

optimizer = optim.Adam(model.parameters(), lr=START_LR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [119]:
class LRFinder:
    def __init__(self, model, optimizer, criterion, device):

        self.optimizer = optimizer
        self.model = model
        self.criterion = criterion
        self.device = device

        torch.save(model.state_dict(), 'init_params.pt')

    def range_test(self, iterator, end_lr=10, num_iter=100,
                   smooth_f=0.05, diverge_th=5):

        lrs = []
        losses = []
        best_loss = float('inf')

        lr_scheduler = ExponentialLR(self.optimizer, end_lr, num_iter)

        iterator = IteratorWrapper(iterator)

        for iteration in range(num_iter):

            loss = self._train_batch(iterator)

            lrs.append(lr_scheduler.get_last_lr()[0])

            # update lr
            lr_scheduler.step()

            if iteration > 0:
                loss = smooth_f * loss + (1 - smooth_f) * losses[-1]

            if loss < best_loss:
                best_loss = loss

            losses.append(loss)

            if loss > diverge_th * best_loss:
                print("Stopping early, the loss has diverged")
                break

        # reset model to initial parameters
        model.load_state_dict(torch.load('init_params.pt'))

        return lrs, losses

    def _train_batch(self, iterator):

        self.model.train()

        self.optimizer.zero_grad()

        x, y = iterator.get_batch()

        x = x.to(self.device)
        y = y.to(self.device)

        y_pred, _ = self.model(x)

        loss = self.criterion(y_pred, y)

        loss.backward()

        self.optimizer.step()

        return loss.item()


class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in
                self.base_lrs]


class IteratorWrapper:
    def __init__(self, iterator):
        self.iterator = iterator
        self._iterator = iter(iterator)

    def __next__(self):
        try:
            inputs, labels = next(self._iterator)
        except StopIteration:
            self._iterator = iter(self.iterator)
            inputs, labels, *_ = next(self._iterator)

        return inputs, labels

    def get_batch(self):
        return next(self)

In [120]:
END_LR = 10
NUM_ITER = 100

lr_finder = LRFinder(model, optimizer, criterion, device)
lrs, losses = lr_finder.range_test(train_iterator, END_LR, NUM_ITER)

RuntimeError: Given input size: (12x1x1). Calculated output size: (12x0x0). Output size is too small